In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

In [3]:
np

<module 'numpy' from '/opt/conda/lib/python3.7/site-packages/numpy/__init__.py'>

In [25]:
# Reading input

articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
# customers = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
# transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [5]:
transactions_train['max_purc_date'] = transactions_train.groupby('customer_id')['t_dat'].transform('max')

transactions_train['min_purc_date'] = transactions_train.groupby('customer_id')['t_dat'].transform('min')

transactions_train['total_visits'] = transactions_train.groupby('customer_id')['t_dat'].transform('nunique')

transactions_train['total_item_bought'] = transactions_train.groupby('customer_id')['article_id'].transform('nunique')

transactions_train['max_year'] = transactions_train['max_purc_date'].str.slice(0,4).astype(int)

In [151]:
transactions_train_filt = transactions_train[transactions_train['max_year'] == 2020]

# transactions_train_filt['product_reach'] = transactions_train_filt.groupby('article_id')['customer_id'].transform('nunique')

# transactions_train_filt = transactions_train_filt[transactions_train_filt['product_reach'] > 8]

transactions_train_filt['repeat_purchase'] = transactions_train_filt.groupby(['customer_id','article_id'])['t_dat'].transform('nunique')

transactions_train_filt = transactions_train_filt[transactions_train_filt['repeat_purchase'] > 1]

# transactions_train_filt = transactions_train_filt[transactions_train_filt['total_visits'] >= 2]

transactions_train_filt = transactions_train_filt[transactions_train_filt['total_item_bought'] >= 12]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [152]:
transactions_train_filt.shape

(2485576, 11)

In [153]:
transactions_train_filt['t_dat_datetime']  = pd.to_datetime(transactions_train_filt['t_dat'])

In [154]:
product_purchase_cycle = transactions_train_filt[['customer_id','article_id','t_dat_datetime']].\
sort_values(['customer_id','article_id','t_dat_datetime']).drop_duplicates()

In [155]:
product_purchase_cycle['t_dat_datetime_lag'] = product_purchase_cycle.\
groupby(['customer_id','article_id'])['t_dat_datetime'].shift(1)

In [156]:
product_purchase_cycle['purchase_cycle'] = product_purchase_cycle['t_dat_datetime'] - product_purchase_cycle['t_dat_datetime_lag']

In [157]:
Average_Purchase_Cycle = product_purchase_cycle[~(product_purchase_cycle['purchase_cycle'].isnull())].\
groupby(['customer_id','article_id']).agg({"purchase_cycle":"median"}).reset_index()

In [158]:
transactions_train_filt = pd.merge(transactions_train_filt,
                                   Average_Purchase_Cycle,
                                   left_on = ['customer_id','article_id'],
                                   right_on = ['customer_id','article_id'],
                                   how='inner'
                                  )

In [159]:
transactions_train_filt.shape

(2485576, 13)

In [160]:
transactions_train_filt['product_last_purchase_date'] = transactions_train_filt.\
groupby(['customer_id','article_id'])['t_dat_datetime'].transform("max")

In [161]:
from datetime import datetime
from datetime import timedelta

In [162]:
transactions_train_filt['purchase_cycle_days'] = transactions_train_filt['purchase_cycle'].astype('str').str.split(' ', expand=True)[0]

In [163]:
transactions_train_filt['next_purchase_date'] = transactions_train_filt['product_last_purchase_date'] + transactions_train_filt['purchase_cycle_days'].astype(int).map(timedelta)

In [164]:
start_date = transactions_train_filt['t_dat_datetime'].max() + timedelta(-90)
end_date = transactions_train_filt['t_dat_datetime'].max() + timedelta(90)

In [165]:
datetime.date(transactions_train_filt['t_dat_datetime'].max()) 

datetime.date(2020, 9, 22)

In [166]:
start_date, end_date

(Timestamp('2020-06-24 00:00:00'), Timestamp('2020-12-21 00:00:00'))

In [167]:
len(transactions_train_filt['customer_id'].unique())

285320

In [144]:
customer_next_purchase = transactions_train_filt[transactions_train_filt['next_purchase_date'].\
                        between(start_date, end_date, inclusive="both")]\
[['customer_id','t_dat','article_id']].drop_duplicates()

In [168]:
customer_next_purchase.shape

(343954, 5)

In [169]:
customer_next_purchase['repeat_purchase'] = customer_next_purchase.groupby(['customer_id'])['article_id'].transform('count')

In [170]:
customer_next_purchase['product_imp_rank'] = customer_next_purchase.groupby(['customer_id'])['repeat_purchase'].rank(method="first",ascending=True)

In [173]:
customer_next_purchase['article_id'] = ' 0'+customer_next_purchase['article_id'].astype('str')

In [179]:
top_12_products = pd.DataFrame(customer_next_purchase[customer_next_purchase['product_imp_rank']<=12]\
                               [['customer_id','article_id']].drop_duplicates().\
                               groupby(['customer_id']).article_id.sum().reset_index())

In [182]:
top_products = ['0706016001', 
'0706016002',
'0372860001',
'0399223001',
'0464297007',
'0160442007',
'0610776002',
'0156231001',
'0399256001',
'0372860002',
'0706016003',
'0759871002']

In [184]:
sample_submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')\
[['customer_id']]

In [188]:
sample_submission = pd.merge(sample_submission,
                             customer_next_purchase[customer_next_purchase['product_imp_rank']<=12]\
                             [['customer_id','article_id']].\
                             drop_duplicates(),
                             left_on = ['customer_id'],
                             right_on = ['customer_id'],
                             how = 'left'
                            )

In [189]:
sample_submission.head()

,customer_id,article_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0723529001
2,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0351484002
3,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN
4,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN


In [ ]:
submission_train_customer = pd.merge(sample_submission['customer_id'], 
                                     transactions_train_filt,
                                     left_on = 'customer_id',
                                     right_on = 'customer_id',
                                     how = 'inner'
                                    )

In [ ]:
import gc
del sample_submission
gc.collect()

In [ ]:
del transactions_train
gc.collect()

In [ ]:
submission_train_customer =  pd.merge(   submission_train_customer,
          customers[['customer_id','age']],
                                     left_on = 'customer_id',
                                     right_on = 'customer_id',
                                     how = 'inner'
                                    )

In [ ]:
age_wise_top_products = submission_train_customer.groupby(['age','article_id']).agg({'customer_id':'nunique'}).reset_index()

age_wise_top_products['rank'] = age_wise_top_products.groupby('age')['customer_id'].rank(method="first",ascending=True)

selected_products = age_wise_top_products[age_wise_top_products['rank']<= 12][['age','article_id']]

In [ ]:
selected_products['article_id'] = ' 0'+selected_products['article_id'].astype('str')

In [ ]:
selected_products['age'].unique()

In [ ]:
age_wise_top_12_products = pd.DataFrame(selected_products.groupby(['age']).article_id.sum().reset_index())

In [ ]:
sample_submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [ ]:
sample_submission.shape

In [ ]:
sample_submission = pd.merge(sample_submission['customer_id'], customers[['customer_id','age']]
         ,left_on = 'customer_id'
         ,right_on = 'customer_id'
         ,how = 'left'
        )

In [ ]:
sample_submission.shape

In [ ]:
sample_submission = pd.merge(sample_submission,
         age_wise_top_12_products,
         left_on = 'age',
         right_on = 'age',
         how = 'left'
        )

In [ ]:
sample_submission['article_id'].fillna( "0706016001 0706016002 0372860001 0399223001 0464297007 0160442007 0610776002 0156231001 0399256001 0372860002 0706016003 0759871002", inplace=True)

In [ ]:
sample_submission.drop('age', axis=1, inplace=True)

In [ ]:
sample_submission = sample_submission.rename(columns = {'article_id':'prediction'})

In [ ]:
sample_submission.to_csv('submission.csv',index=False)